In [1]:
import requests
import dotenv
import os
import numpy
import geopandas as gpd
import pandas as pd
import datetime
import urllib.parse as urlparse
import shapely

In [2]:
dotenv.load_dotenv()
api_key = os.environ.get('API_KEY')

In [3]:
def getHospitalRangesDf(df, depart_datetimes, travel_times, api_key):
    buf_data = {}
    buf_out = []
    hospitals = df['hospital_name'].unique()
    for hospital in hospitals:
        for depart_datetime in depart_datetimes:
            for travel_time in travel_times:
                coords_str = '{0},{1}'.format(df[df['hospital_name'] == hospital].y_coord.values[0], df[df['hospital_name'] == hospital].x_coord.values[0])
                depart_datetime_str = depart_datetime.strftime('%Y-%m-%dT%H:%M:%S')
                api_call = 'https://api.tomtom.com/routing/1/calculateReachableRange/'+ urlparse.quote(coords_str) +'/json?timeBudgetInSec='+ str(travel_time) +'&departAt='+ urlparse.quote(depart_datetime_str) +'&traffic=true&routeType=fastest&travelMode=van&vehicleMaxSpeed=120&vehicleWeight=4500&vehicleLength=5.99&vehicleWidth=1.92&vehicleHeight=2.32&vehicleCommercial=false&vehicleEngineType=combustion&key='+api_key
                api_request = requests.get(api_call)
                api_response = api_request.json()
                buf_data['from_hospital'] = hospital
                buf_data['time_of_depart'] = depart_datetime_str
                buf_data['is_public'] = df[df['hospital_name'] == hospital].is_public.values[0]
                buf_data['travel_time'] = travel_time
                buf_data['hosp_x_coord'] = api_response['reachableRange']['center']['longitude']
                buf_data['hosp_y_coord'] = api_response['reachableRange']['center']['latitude']
                buf_data['reachable_range'] = shapely.geometry.Polygon([[coords['longitude'], coords['latitude']] for coords in api_response['reachableRange']['boundary']])
                buf_out.append(buf_data)
                buf_data = {}
    out_df = gpd.GeoDataFrame(buf_out, geometry='reachable_range', crs='EPSG:4326')
    out_df['time_of_depart'] = pd.to_datetime(out_df.time_of_depart)
    return out_df

In [4]:
hospital_df = pd.read_csv('hospital_coords.csv')
hospital_df.loc[34] = ['South City Hospital and Medical Center', 0, 121.00087947, 14.37562411] # Had to add this manually as the hospital isn't in OSM
hospital_df

,hospital_name,is_public,x_coord,y_coord
0,DasmariÃ±as City Medical Center,0,120.981590,14.353421
1,Binakayan Hospital and Medical Center,0,120.926088,14.451960
2,Prime Global Care Medical Center Inc.,0,120.974338,14.397959
3,Kawit Maternity and General Hospital,0,120.905960,14.445855
4,Asia Medic Family Hospital & Medical Center,0,120.956559,14.298706
5,Pamplona Hospital and Medical Center,0,120.968355,14.463625
6,Golden Acres Hospital Center,0,120.997669,14.419037
7,A. Zarate General Hospital,0,120.980443,14.468559
8,Alabang Medical Center,0,121.024479,14.427765
9,Las PiÃ±as General Hospital & Satellite Trauma...,1,120.973806,14.471637


In [5]:
# Chose 4 dates of interest to check area of coverage
# 2019-6-19 2AM  - no traffic              2019-6-19 5PM  - high traffic
# 2019-12-23 2AM - event day no traffic    2019-12-23 5PM - event day high traffic

depart_datetimes = [datetime.datetime(2019, 6, 19, 2, 0, 0), datetime.datetime(2019, 6, 19, 17, 0, 0),
                    datetime.datetime(2019, 12, 23, 2, 0, 0), datetime.datetime(2019, 12, 23, 17, 0, 0)]

# Chose 2 travel times of interest
# 480 secs (8 minutes) is standard international response time from (Estember, Isip, & Misal, 2019)
# 1,380 secs (23 minutes) is average response time of EMS responders in PH (Estember, Isip, & Misal, 2019)
travel_times = [480, 1380]

In [6]:
hosp_ranges_df = getHospitalRangesDf(hospital_df, depart_datetimes, travel_times, api_key)
hosp_ranges_df.to_file('hosp_ranges_df.geojson', index=False, crs='EPSG:4326')

In [7]:
hosp_ranges_df = gpd.read_file('hosp_ranges_df.geojson')
hosp_ranges_df

,from_hospital,time_of_depart,is_public,travel_time,hosp_x_coord,hosp_y_coord,geometry
0,DasmariÃ±as City Medical Center,2019-06-19 02:00:00,0,480,120.98160,14.35304,"POLYGON ((120.97769 14.39348, 120.97318 14.387..."
1,DasmariÃ±as City Medical Center,2019-06-19 02:00:00,0,1380,120.98160,14.35304,"POLYGON ((120.96906 14.45573, 120.96001 14.463..."
2,DasmariÃ±as City Medical Center,2019-06-19 17:00:00,0,480,120.98160,14.35304,"POLYGON ((120.97858 14.38163, 120.97800 14.380..."
3,DasmariÃ±as City Medical Center,2019-06-19 17:00:00,0,1380,120.98160,14.35304,"POLYGON ((120.97418 14.41401, 120.96815 14.412..."
4,DasmariÃ±as City Medical Center,2019-12-23 02:00:00,0,480,120.98160,14.35304,"POLYGON ((120.97770 14.39258, 120.97356 14.387..."
...,...,...,...,...,...,...,...
275,South City Hospital and Medical Center,2019-06-19 17:00:00,0,1380,121.00071,14.37495,"POLYGON ((120.99710 14.41817, 120.99084 14.417..."
276,South City Hospital and Medical Center,2019-12-23 02:00:00,0,480,121.00071,14.37495,"POLYGON ((120.99937 14.39482, 120.99618 14.393..."
277,South City Hospital and Medical Center,2019-12-23 02:00:00,0,1380,121.00071,14.37495,"POLYGON ((120.99566 14.44342, 120.99084 14.417..."
278,South City Hospital and Medical Center,2019-12-23 17:00:00,0,480,121.00071,14.37495,"POLYGON ((120.99937 14.39482, 120.99618 14.393..."


In [8]:
overlay = gpd.read_file('overlay_bacoor.geojson')
overlay

,fid,GID_0,NAME_0,GID_1,NAME_1,NL_NAME_1,GID_2,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,419,PHL,Philippines,PHL.24_1,Cavite,None,PHL.24.3_1,Bacoor,None,None,Bayan|Munisipyo,Municipality,42103,PH.CV.BC,"MULTIPOLYGON (((120.98454 14.36196, 120.97490 ..."


In [9]:
for depart_datetime in depart_datetimes:
    gpd.clip(hosp_ranges_df[(hosp_ranges_df['travel_time'] == 480) & (hosp_ranges_df['time_of_depart'] == depart_datetime)].dissolve(), overlay).to_file('standard_time_'+ depart_datetime.strftime('%Y%m%d%H') +'.geojson')

C:\ProgramData\Anaconda3\envs\traffic\lib\site-packages\geopandas\tools\clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
C:\ProgramData\Anaconda3\envs\traffic\lib\site-packages\geopandas\tools\clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[
C:\ProgramData\Anaconda3\envs\traffic\lib\site-packages\geopandas\tools\clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.